# <center> Etapa 2
---

### <center> 1. Proceso de automatización del proceso de preparación de datos, construcción del modelo, persistencia del modelo y acceso por medio de API

#### 1.1 Creación de PipeLine

##### Librerias

In [24]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

##### Cargar Datos

In [25]:
df = pd.read_csv("fake_news_spanish.csv", delimiter=';')

In [26]:
df.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


##### Datos Entrenamiento 

In [27]:
df_train=df.copy()
df_train.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


En este punto se crea una copia del DataFrame y se eliminan las filas con valores nulos en las columnas de Titulo y Descripic+on. Posteriormente se combinan estas dos columnas en una llamada texto el cual es el que se va a usar para poder detectar las noticias falsas. La variable X contiene el texto de la noticia, mientras que y contiene las etiquetas (1=fake, 0=real) la cual es la variable objetivo del problema

In [28]:
df_train = df_train.dropna(subset=['Titulo', 'Descripcion'])
df_train['texto'] = df_train['Titulo'].fillna('') + " " + df_train['Descripcion'].fillna('')
X = df_train['texto']
y = df_train['Label']


In [29]:
df_train.head()

,ID,Label,Titulo,Descripcion,Fecha,texto
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,'The Guardian' va con Sánchez: 'Europa necesit...
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,El 'Ahora o nunca' de Joan Fuster sobre el est...
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,Puigdemont: 'No sería ninguna tragedia una rep...


##### Dividir en conjunto de entrenamiento y prueba

Se dividen los datos en conjunto de entrenamiento y prueba, la divisóm se hace 80% para entrenamiento y 20% para prueba, esto se hace ya que si se entrena el modelo con todos los datos sin reservar un conjunto de prueba, no se podría evaluar como se desempanará el modelo con datos nuevos. La idea es que el modelo aprenda patrones en el 80% de los datos y luego los apliquen al 20% restante que no ha visto. Además permite, eviatr el sobreajuste, pues si un modelo se ajusta demasiado a los datos de entrenamiento, puede memorizar en lugar de aprender patrones generales. Además, el random_state=42 asegura que la divisón sea reproducible. 

In [30]:
# Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##### Crear Pipeline con vectorización  TF-IDF y Regresión Logistica

El pipeLine encapsula la transfromación de texto utilizando vectorización y el modelo elegido para este que es la Regresikón Logística, ya que es el que mostro mayor presición con la detección de noticias falsas. La parte de TfidVectorizaer convierte texto en valores de TF-IDF que es requerido para el modelo, pero lo hace en un solo paso y lo va aplicando a los datos que van llegando al modelo. Para el modelo se ajusta el modelo para los datos desbalanciados y se pone un max_inter=1000 que evita que el modelo se quede sin converger. 

In [ ]:
# Crear pipeline con vectorización TF-IDF y Regresión Logística
from sklearn.linear_model import LogisticRegression


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(use_idf=True, lowercase=True)),
    ('classifier', LogisticRegression(class_weight='balanced', max_iter=1000))
])

Ahora se entrena el modelo utilizando los datos de entrenamiento que contienen la variable objetivo que en este caso en específico es la variable label con valores 0 y 1.

In [44]:
# Entrenar el modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

Una vez se entrena el modelo, se debe evaluar que tan bien está clasificando las noticias falsas. En este caso se puede ver que el valor de Accuracy es de 89.82% lo que indica que el modelo clasifica correctamente casi el 90% de las instancias en el conjunto de validación. Ahora bien, viendo el reporte de clasificación este evalua el desempeño del modelo en términos de precisión, recall y puntaje F1 para cada clase. En este caso la clase 0 (negativa) tiene una presición de 0.92 lo que indica que el 92% de los ejemplos que el modelo predice como clase 0 realmente pertenecen a la clase 0. Exhaustividad (recall) es de 0.83 lo que muestra que el 83% de los ejemplos reales de la calse 0 fueron detectatos correctamente. Por otro lado, la clase 1 (positiva) muestra que la presicipon es del 89% lo que muestra que los ejemplos predichos como clase 1 realmente pertenecen a esta clase. El recall es de 95% lo que dice que los jemeplos reales de clase 1 fueron detecta dos correctamente. En conclusión, el modelo tiene un buen desempeño general con una exactitud del 89.82%, este detecta muy bien la calse 1 pero tiene un poco de dificultaddes co la clae 0. Por otro lado, muestra que el equilibrio entre presición y recall es sólido, lo que indica que el modelo no favorece demasiado a una clase sobre otra. Todo lo anterior indica que el modelo funciona lo suficientemente bien para lo que se desea realizar. 

In [45]:
# Evaluar el modelo en el conjunto de validación
y_pred = pipeline.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy en conjunto de validación: {accuracy:.4f}")
print("Reporte de clasificación:\n", classification_report(y_val, y_pred))


Accuracy en conjunto de validación: 0.8982
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.92      0.83      0.87      4706
           1       0.89      0.95      0.92      6704

    accuracy                           0.90     11410
   macro avg       0.90      0.89      0.89     11410
weighted avg       0.90      0.90      0.90     11410



##### Reentrenar con todos los datos disponibles

En este punto se entrena nuevamente el modelo, pero ahora con todos los datos, este se hace para poder mejorar el desempeño antes de aplicarlo a datos nuevos. 

In [46]:
# Volver a entrenar con todos los datos disponibles
pipeline.fit(X, y)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier',
                 LogisticRegression(class_weight='balanced', max_iter=1000))])

##### Carga y procesamiento de datos de prueba


En esta oarte se realiza la carga de datos de notiicias que no tiene la etiqueta de label que es la variable que se debe predecir, para estos se aplica el mismo preprocesamiento hecho en la anterior carga. 

In [47]:

# Cargar datos de prueba (sin etiquetas) para predicción
df_test = pd.read_csv("fake_news_test.csv", delimiter=';')
df_test = df_test.dropna(subset=['Titulo', 'Descripcion'])
df_test['texto'] = df_test['Titulo'].fillna('') + " " + df_test['Descripcion'].fillna('')
X_test = df_test['texto']


##### Ajuste del umbral de predicción

Se hace un ajuste del umbral, se pone predict_proba(X_test)[:, 1] que obtiene la probabilidad de que una noticia sea falsa, es decri que pertenezca a la clase 1. Por otro lado se establece un umbral del 60% para mejorar el balance entre precisión y recall

In [48]:
# Ajustar umbral de predicción con predict_proba
probabilidades = pipeline.predict_proba(X_test)[:, 1]
umbral = 0.6  # Ajustar el umbral para mejorar el balance de recall



##### Clasificación según el umbral ajustado

Esta parte determina si la probabilidad es mayor o igual al umbral, se calsifica como 1 es decir una Fake news. Por otro lado, si es menor se claasifica como 0 es decir una noticia real. 

In [49]:

# Clasificar según el umbral ajustado
df_test['Prediccion'] = (probabilidades >= umbral).astype(int)

##### Guardar el modelo entrenado

In [50]:
# Guardar las predicciones en un archivo CSV
df_test[['Titulo', 'Descripcion', 'Prediccion']].to_csv("fake_news_test_predicciones.csv", index=False)


In [51]:
# Guardar el modelo entrenado
joblib.dump(pipeline, "modelo_fake_news.joblib")

print("Modelo entrenado y guardado como 'modelo_fake_news.joblib'")
print("Predicciones guardadas en 'fake_news_test_predicciones.csv'")

Modelo entrenado y guardado como 'modelo_fake_news.joblib'
Predicciones guardadas en 'fake_news_test_predicciones.csv'
